In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
import json

conf = SparkConf()

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("test")
         .getOrCreate())

In [3]:
spark

In [4]:
# смотрю каталог
!hdfs dfs -ls /labs/laba01/ml-100k

Found 23 items
-rw-r--r--   3 hdfs hdfs       6750 2022-01-06 18:46 /labs/laba01/ml-100k/README
-rw-r--r--   3 hdfs hdfs        716 2022-01-06 18:46 /labs/laba01/ml-100k/allbut.pl
-rw-r--r--   3 hdfs hdfs        643 2022-01-06 18:46 /labs/laba01/ml-100k/mku.sh
-rw-r--r--   3 hdfs hdfs    1979173 2022-01-06 18:46 /labs/laba01/ml-100k/u.data
-rw-r--r--   3 hdfs hdfs        202 2022-01-06 18:46 /labs/laba01/ml-100k/u.genre
-rw-r--r--   3 hdfs hdfs         36 2022-01-06 18:46 /labs/laba01/ml-100k/u.info
-rw-r--r--   3 hdfs hdfs     236344 2022-01-06 18:46 /labs/laba01/ml-100k/u.item
-rw-r--r--   3 hdfs hdfs        193 2022-01-06 18:46 /labs/laba01/ml-100k/u.occupation
-rw-r--r--   3 hdfs hdfs      22628 2022-01-06 18:46 /labs/laba01/ml-100k/u.user
-rw-r--r--   3 hdfs hdfs    1586544 2022-01-06 18:46 /labs/laba01/ml-100k/u1.base
-rw-r--r--   3 hdfs hdfs     392629 2022-01-06 18:46 /labs/laba01/ml-100k/u1.test
-rw-r--r--   3 hdfs hdfs    1583948 2022-01-06 18:46 /labs/laba01/ml-100k/u2.base


In [5]:
# смотрим мой вариант
rdd_film = spark.sparkContext.textFile("/labs/laba01/ml-100k/u.item")
rdd_film = rdd_film.map(lambda x: x.split("|"))
rdd_film.filter(lambda x: x[0] == '96').collect()

[['96',
  'Terminator 2: Judgment Day (1991)',
  '01-Jan-1991',
  '',
  'http://us.imdb.com/M/title-exact?Terminator%202:%20Judgment%20Day%20(1991)',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '1',
  '0',
  '0']]

# Ratings

In [6]:
rdd_df = spark.sparkContext.textFile("/labs/laba01/ml-100k/u.data")

In [7]:
rdd_df = rdd_df.map(lambda x: x.split("\t"))

In [8]:
rdd_df.take(5)

[['196', '242', '3', '881250949'],
 ['186', '302', '3', '891717742'],
 ['22', '377', '1', '878887116'],
 ['244', '51', '2', '880606923'],
 ['166', '346', '1', '886397596']]

In [9]:
my_film = rdd_df.filter(lambda x: x[1] == '96')

In [10]:
my_film = rdd_df.filter(lambda x: x[1] == '96')

In [11]:
my_film.take(5)

[['42', '96', '5', '881107178'],
 ['193', '96', '1', '889124507'],
 ['200', '96', '5', '884129409'],
 ['197', '96', '5', '891409839'],
 ['263', '96', '4', '891298336']]

In [12]:
film_score = sorted(my_film.map(lambda x: (x[2],1)).countByKey().items())

In [13]:
all_score = sorted(rdd_df.map(lambda x: (x[2],1)).countByKey().items())

In [14]:
result = {'hist_film':[item[1] for item in film_score],'hist_all':[item[1] for item in all_score]}

In [15]:
with open('lab01_1.json', 'w') as file:
    json.dump(result, file)

In [16]:
spark.stop()